In [1]:
import numpy as np
from matplotlib import pyplot as plt

# 23.6

In [2]:
def my_LQG(A,B,H,Q,M,V,W,R,x0,T):
    S = [None]*(T+1)
    P = [None]*(T+1)
    #setup the LQR gain here
    P[T] = M
    for k in range(T, -1, -1):
        var = R+(B.T@P[k])@B
        P[k-1] = (A.T@P[k])@A + Q - (A.T@P[k]@B)@np.linalg.pinv(var)@(B.T@P[k]@A)

    u = [None]*T
    K = [None]*(T+1)
    L = [None]*T
    S = [None]*(T+1)
    S[0] = np.eye(2)
    #setup the Kalman gain
    for k in range(1,T):
        S[k] = A@(S[k-1]-S[k-1]@H.T@np.linalg.pinv(H@S[k-1]@H.T+ V)@H@S[k-1])@A.T+W
    #now compute both gain matrices at each step
    #note that np.squeeze is only used because this term is - actually a scalar...
    K[0] = -B.T@P[1]@A/(np.squeeze(B.T@P[1]@B)+R)
    for k in range(1,T):
        K[k] = -B.T@P[k+1]@A/(np.squeeze(B.T@P[k+1]@B)+R)
        L[k] = S[k-1]@H.T@np.linalg.pinv(H@S[k-1]@H.T+V)
    full_state = [None]*(T+1)
    full_state[0] = np.array([x0]).T
    observed_state = [None]*(T+1)
    #finally we are ready to compute the solution.
    u[0] = K[0]@full_state[0]
    for k in range(1,T):
        full_state[k] = A@full_state[k-1]+B@u[k-1] + np.random.multivariate_normal(np.zeros(len(W)), cov=W)
        observed_state[k] = H@full_state[k] + np.random.multivariate_normal(np.zeros(len(V)), cov=V)
        u[k] = -K[k]@observed_state[k]
    return u, observed_state, full_state

# 23.7

In [3]:
A = np.array([[1,1],[0,1]])
B = np.array([[0],[1]])
H = np.diag([1,0])
Q, M = np.eye(2), np.eye(2)
V, W = 10*np.eye(2), 20*np.eye(2)
x0 = np.array([1,0])
R = 1

u, obs, state = my_LQG(A, B, H, Q, M, V, W, R, x0, 100)
print(u)
print(obs)
print(state)

[array([[-0.42208244]]), array([[11.05791817,  6.72744554]]), array([[ 3.56213543, 13.51208094]]), array([[-6.79720415, 11.6345439 ]]), array([[ 4.22882792, 34.0458497 ]]), array([[13.69830291, 51.11489629]]), array([[ 5.29042245, 82.17002056]]), array([[ 13.17908031, 140.39172298]]), array([[ 22.56477173, 216.84399701]]), array([[ 35.79638076, 382.02091394]]), array([[ 58.80352018, 614.17129688]]), array([[ 112.81762212, 1016.2404102 ]]), array([[ 178.90406906, 1677.90824047]]), array([[ 298.30860403, 2763.33345001]]), array([[ 485.71235771, 4559.80015063]]), array([[ 798.4633395 , 7536.34804196]]), array([[ 1317.58105904, 12412.50276661]]), array([[ 2172.55376995, 20496.25497531]]), array([[ 3587.03080833, 33807.58059886]]), array([[ 5905.29352593, 55761.46497655]]), array([[ 9752.54202612, 92006.59007956]]), array([[ 16085.62960019, 151774.23932839]]), array([[ 26538.21202078, 250375.37267798]]), array([[ 43779.2665977 , 413038.29921685]]), array([[ 72227.23393418, 681386.29047495]]